In [1]:
# Import required libraries for data manipulation and analysis
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr

In [2]:
#Import required sklearn functions
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn.inspection import permutation_importance
from collections import defaultdict

In [3]:
#Import sklearn classifiers
from sklearn.ensemble import RandomForestClassifier

In [4]:
#Import library to oversample 
from imblearn.over_sampling import RandomOverSampler

In [5]:
#Import RDKit and Mordred libraries
from rdkit import Chem
from rdkit.Chem import Draw
from mordred import Calculator, descriptors

In [6]:
# Sets Pandas Display to Monitor Code
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)

In [7]:
# Create Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True)

In [8]:
#Functions used in the study

#Remove those numbers from analysis data
def filter_rows_by_values1(df, col, values):
    return df[~df[col].isin(values)]

#Remove those numbers from analysis data
def filter_rows_by_values2(df, col, values):
    return df[df[col].isin(values)]

#Get Mordred calcs
def get_Mordred(data_input):
    # Assigns Reactants Mordred Info
    reactants = data_input['Substrate']
    
    reactants_mol_list = []
    for inChi_reactants in reactants:
      reactants_mol = Chem.MolFromInchi(inChi_reactants)
      reactants_mol_list.append(reactants_mol)

    # Puts reactants into Pandas Type
    reactant_data = []
    reactant_data = calc.pandas(reactants_mol_list)
       
    #Joins Mordred parameters with experimental, atomic charges, and JChem for Excel parameters
    add_reactants = pd.concat((data_input, reactant_data), axis=1)
    
    #Force any non-numeric entries as NaN and replace them with 0
    int_data = add_reactants.apply(pd.to_numeric, errors='coerce')
    
    output = int_data.fillna(0)#, inplace=True)

    return output

#Remove zero varience
def remove_zero_varience(values):
   sel = VarianceThreshold()
   _ = sel.fit(values)
   mask = sel.get_support()
   values = values.loc[:,mask] 
   return values

def remove_95correlated(correlated):
    #Remove any features that are greater than 95% correlated
    corr_matrix = correlated.corr()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

    correlated = correlated.drop(to_drop, axis = 1)
    corr_matrix = correlated.corr()
    return correlated

def remove_nonimportant(X_values, y_values):
    # Specifys Random Forest and the Number of Trees, SelectFromModel will
    # select features which are most important
    feature_names = [f"feature {i}" for i in range(X_values.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(X_values, y_values)

    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time

    threshold = np.sort(importances)[-100]
    
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(X_values, y_values)

    # Select the final features set 
    sel.get_support()
    selected_feat= X_values.columns[(sel.get_support())]

    # Prints the names of the final selected features
    print(selected_feat)
    X_values = X_values[selected_feat]
    
    return X_values

def dendrogram(X_values, y):
    corr = spearmanr(X_values).correlation
    # Ensure the correlation matrix is symmetric
    corr = (corr + corr.T) / 2
    np.fill_diagonal(corr, 1)
    distance_matrix = 1 - np.abs(corr)
    dist_linkage = hierarchy.ward(squareform(distance_matrix))
  
    trained_cluster_ids = hierarchy.fcluster(dist_linkage, y, criterion="distance")
    trained_cluster_id_to_feature_ids = defaultdict(list) 
    for idx, trained_cluster_id in enumerate(trained_cluster_ids):
        trained_cluster_id_to_feature_ids[trained_cluster_id].append(idx)
    
    trained_selected_features = [v[0] for v in trained_cluster_id_to_feature_ids.values()]
    final_selected_features = X_values.columns[trained_selected_features]
    X_train = X_values[final_selected_features]
    return X_train

def classificationMetrics(results, y_test, pred):
    acc = accuracy_score(y_test, pred)
    prec = precision_score(y_test, pred, average=None, zero_division=0)
    recall = recall_score(y_test, pred, average=None)
    F1 = f1_score(y_test, pred, average=None)           
    #Calculate confusion matrix
    cf_matrix = confusion_matrix(y_test, pred)
    cf_matrix = np.reshape(cf_matrix,(1,4))
    comb = np.concatenate((x, y, cf_matrix, acc, prec, recall, F1), axis=None)
    comb = [comb]
    results = results.append(pd.DataFrame(comb, columns=results.columns), ignore_index=True)
    return results


In [9]:
# Read Training/Test data input File
data = pd.read_csv('BorylationTrainingTest 8-29-24.csv')

#group the compounds by numbers
data['grouped'] = data.groupby('Substrate', sort=False).ngroup()

data = data[['Substrate','Product_Ratio','grouped','Boronic Ester','Active Catalyst-Ligand','Catalyst','Ligand','Buried_Vol',
       'PyramidalizationAR','PyramidalizationG','SASA_area','SASA_vol','Sterimol_L','SterimolB_1','Sterimol_B_5','Buried_Sterimol_L',
       'Buried_SterimolB_1','Buried_Sterimol_B_5','Hirshfeld Heavy Atom Charge','CM5 Charge','Hirshfeld Carbon Charge','Hirshfeld Hydrogen Charge',
       'ESP Heavy Atom Charge','ESP Carbon Charge','ESP Hydrogen Charge','NPA Carbon Charge','NPA Hydrogen Charge','MBS Heavy Atom Charge',
       'MBS Carbon Charge','MBS Hydrogen Charge','Mulliken Heavy Charge','Mulliken Carbon Charge','Mulliken Hydrogen Charge','Solvent','Temp']]

#Convert substrates to Mordred features
data = get_Mordred(data)

data.head()

100%|██████████| 1033/1033 [01:27<00:00, 11.82it/s]


,Substrate,Product_Ratio,grouped,Boronic Ester,Active Catalyst-Ligand,Catalyst,Ligand,Buried_Vol,PyramidalizationAR,PyramidalizationG,SASA_area,SASA_vol,Sterimol_L,SterimolB_1,Sterimol_B_5,Buried_Sterimol_L,Buried_SterimolB_1,Buried_Sterimol_B_5,Hirshfeld Heavy Atom Charge,CM5 Charge,Hirshfeld Carbon Charge,Hirshfeld Hydrogen Charge,ESP Heavy Atom Charge,ESP Carbon Charge,ESP Hydrogen Charge,...,MWC05,MWC06,MWC07,MWC08,MWC09,MWC10,TMWC10,SRW02,SRW03,SRW04,SRW05,SRW06,SRW07,SRW08,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0.0,1,0,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.79083,7.558608,5.941608,4.79083,7.2781,-0.003389,-0.000578,-0.101759,0.032223,-0.078389,-0.182602,0.032537,...,5.129899,5.758902,6.385194,7.016610,7.645398,8.277158,66.911677,2.708050,0.0,3.663562,0.0,4.762174,0.0,5.926926,0.0,7.126891,32.187603,114.140851,4.390033,84,5,26.0,24.0,3.500000,2.25
1,0.0,0,0,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.79083,7.558608,5.941608,4.79083,7.2781,0.003094,0.000249,-0.057714,0.030404,0.098246,0.182934,-0.042344,...,5.129899,5.758902,6.385194,7.016610,7.645398,8.277158,66.911677,2.708050,0.0,3.663562,0.0,4.762174,0.0,5.926926,0.0,7.126891,32.187603,114.140851,4.390033,84,5,26.0,24.0,3.500000,2.25
2,0.0,0,0,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.79083,7.558608,5.941608,4.79083,7.2781,-0.000052,-0.000032,-0.059257,0.029602,-0.011526,0.182934,-0.012016,...,5.129899,5.758902,6.385194,7.016610,7.645398,8.277158,66.911677,2.708050,0.0,3.663562,0.0,4.762174,0.0,5.926926,0.0,7.126891,32.187603,114.140851,4.390033,84,5,26.0,24.0,3.500000,2.25
3,0.0,0,0,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.79083,7.558608,5.941608,4.79083,7.2781,0.000299,0.000312,-0.058954,0.029626,-0.008331,0.037009,-0.022670,...,5.129899,5.758902,6.385194,7.016610,7.645398,8.277158,66.911677,2.708050,0.0,3.663562,0.0,4.762174,0.0,5.926926,0.0,7.126891,32.187603,114.140851,4.390033,84,5,26.0,24.0,3.500000,2.25
4,0.0,1,1,2,6,2,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.79083,7.558608,5.941608,4.79083,7.2781,-0.001043,0.001811,-0.100893,0.032591,-0.063472,-0.138162,0.018852,...,6.013715,6.748760,7.482682,8.223359,8.962007,9.704610,85.894968,3.218876,0.0,4.290459,0.0,5.533389,0.0,6.867974,0.0,8.259717,41.170416,185.214350,4.630359,300,12,48.0,48.0,5.361111,3.50


In [10]:
Results_df = pd.DataFrame(columns =  ['x', 'y',  "True Neg","False Pos","False Neg","True Pos",'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

maxacc_comb = pd.DataFrame()
val_tot = pd.DataFrame()
prod = pd.DataFrame()
test_index_total = pd.DataFrame()

model_columns = pd.DataFrame()
for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 200,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(data, "grouped", set_numbers)
    test_data = filter_rows_by_values2(data, "grouped", set_numbers)
  
    #Remove features that dont change
    training_data = remove_zero_varience(training_data)
    
    #Remove features that are more than 95% correlated
    training_data = remove_95correlated(training_data)
    
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio', axis = 1)
    training_X = training_X.drop('grouped', axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio', axis = 1)
    test_X = test_X.drop('grouped', axis = 1)
    test_y = test_data['Product_Ratio']
 
    #Remove features that are considered less important
    feature_names = [f"feature {i}" for i in range(training_X.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(training_X, training_y)
    
    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time
    
    threshold = np.sort(importances)[-100] 
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(training_X, training_y)
     
    # Select the reduced features set 
    sel.get_support()
    selected_feat= training_X.columns[(sel.get_support())]
    
    reduced1_X = training_X[selected_feat]
    test_X = test_X[selected_feat]
    
    #Apply over-sampling to dataset
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(reduced1_X, training_y) 
    
    for y in [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
              0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 
              0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35,
              0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 
              0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45,
              0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50,
              0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55]:
             
    
        #Make final training and test set and save them as df's  
        X_train = dendrogram(X_resampled, y)
        test_X = test_X[X_train.columns]
        training_columns_list = X_train.columns.tolist()
        training_columns_list = (x, y, training_columns_list)
        training_columns_list = (pd.DataFrame(training_columns_list).T)

        #Random Forest Classifier
        rfc = RandomForestClassifier(n_estimators=800,max_depth=9)
        rfc.fit(X_train, y_resampled)
        pred = rfc.predict(test_X)
        Results_df = classificationMetrics(Results_df, test_y, pred)

        #Evaluate model by going line by line
        ynew = rfc.predict(test_X)
        prediction_df = pd.DataFrame(ynew,  columns = [(x,y)])

        val_pred_T = prediction_df.T
        val_tot = val_tot.append(val_pred_T)

        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = Results_df.groupby('y')['acc'].mean()
        acc_std = Results_df.groupby('y')['acc'].std()
        precision_0_mean = Results_df.groupby('y')['precision 0'].mean()
        precision_0_std = Results_df.groupby('y')['precision 0'].std()
        precision_1_mean = Results_df.groupby('y')['precision 1'].mean()
        precision_1_std = Results_df.groupby('y')['precision 1'].std()
        recall_0_mean = Results_df.groupby('y')['recall 0'].mean()
        recall_0_std = Results_df.groupby('y')['recall 0'].std()
        recall_1_mean = Results_df.groupby('y')['recall 1'].mean()
        recall_1_std = Results_df.groupby('y')['recall 1'].std()
        F1_0_mean = Results_df.groupby('y')['F1 0'].mean()
        F1_0_std = Results_df.groupby('y')['F1 0'].std()
        F1_1_mean = Results_df.groupby('y')['F1 1'].mean()
        F1_1_std = Results_df.groupby('y')['F1 1'].std()
        true_neg_mean = Results_df.groupby('y')['True Neg'].mean()
        true_neg_std = Results_df.groupby('y')['True Neg'].std()
        false_pos_mean = Results_df.groupby('y')['False Pos'].mean()
        false_pos_std = Results_df.groupby('y')['False Pos'].std()        
        false_neg_mean = Results_df.groupby('y')['False Neg'].mean()
        false_neg_std = Results_df.groupby('y')['False Neg'].std()      
        true_pos_mean = Results_df.groupby('y')['True Pos'].mean() 
        true_pos_std = Results_df.groupby('y')['True Pos'].std()   
        

        average_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std,
                                   true_neg_mean, true_neg_std,
                                   false_pos_mean, false_pos_std,
                                   false_neg_mean, false_neg_std,
                                   true_pos_mean, true_pos_std], axis=1)
    
        average_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std', 'true_neg_mean', 'true_neg_std',
                                 'false_pos_mean', 'false_pos_std','false_neg_mean', 'false_neg_std',
                                 'true_pos_mean', 'true_pos_std']                                

        maxacc = average_df[average_df.acc_mean == average_df.acc_mean.max()]
        maxacc_copy  = maxacc.copy()
        maxacc_copy['x_col'] = x
        
        model_columns = model_columns.append(training_columns_list)
        #print(x,y)
    test_index = pd.DataFrame(test_data.index.values)
    test_index_total = pd.concat([test_index_total, test_index],axis = 1)
    test_y = test_y.rename(x)
    prod = prod.append(test_y)
    maxacc_comb = maxacc_comb.append(maxacc_copy)  

maxacc_comb.to_csv("chargeMordredEXP.csv")
model_columns = model_columns.rename(columns = {0:'x', 1:'y', 2: 'features'})
model_columns = model_columns.drop_duplicates(subset = ['x','y'])
model_columns.to_csv("chargeMordredEXP.csv", mode="a")

In [11]:
maxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,true_neg_mean,true_neg_std,false_pos_mean,false_pos_std,false_neg_mean,false_neg_std,true_pos_mean,true_pos_std,x_col
y,,,,,,,,,,,,,,,,,,,,,,,
0.40,0.854639,0.005852,0.898625,0.004703,0.663110,0.015921,0.920779,0.005122,0.600000,0.020412,0.909557,0.003657,0.629841,0.015872,141.800000,0.788811,12.200000,0.788811,16.000000,0.816497,24.000000,0.816497,1
0.35,0.842628,0.011211,0.911399,0.014803,0.619831,0.034520,0.887709,0.029710,0.671250,0.076638,0.898899,0.009600,0.640540,0.024290,133.700000,7.630272,16.800000,4.073018,13.150000,3.065513,26.850000,3.065513,2
0.35,0.848606,0.012746,0.911136,0.012080,0.624558,0.029835,0.897972,0.028358,0.650833,0.069009,0.904109,0.010913,0.634248,0.022414,142.100000,13.578253,15.900000,3.575298,13.966667,2.760351,26.033333,2.760351,3
0.35,0.840769,0.017787,0.907342,0.012529,0.604751,0.043627,0.891472,0.027113,0.638750,0.063788,0.899030,0.013068,0.618784,0.033890,140.800000,11.936155,16.950000,3.615848,14.450000,2.551520,25.550000,2.551520,4
0.35,0.840141,0.016660,0.907053,0.011488,0.603644,0.040742,0.890710,0.024697,0.640000,0.057809,0.898557,0.012147,0.619243,0.031829,139.980000,10.801531,17.020000,3.310404,14.400000,2.312345,25.600000,2.312345,5
0.35,0.836526,0.017718,0.907907,0.010841,0.590773,0.047915,0.885130,0.026095,0.642917,0.053696,0.896102,0.012686,0.613465,0.032759,140.366667,9.904573,18.133333,3.980745,14.283333,2.147854,25.716667,2.147854,6
0.35,0.835181,0.016809,0.906261,0.010879,0.579784,0.052173,0.886250,0.024423,0.625714,0.065659,0.895895,0.011803,0.599814,0.045484,143.914286,12.677428,18.371429,3.757780,14.971429,2.626352,25.028571,2.626352,7
0.35,0.826421,0.028259,0.908047,0.011377,0.564914,0.062936,0.871686,0.045093,0.640937,0.073803,0.888707,0.022188,0.596099,0.043864,140.550000,14.859596,20.450000,6.569781,14.362500,2.952123,25.637500,2.952123,8
0.35,0.824796,0.027128,0.907216,0.011147,0.557810,0.062870,0.870762,0.042594,0.635278,0.071882,0.887914,0.021078,0.590065,0.045215,141.433333,14.226143,20.788889,6.270126,14.588889,2.875290,25.411111,2.875290,9


In [12]:
selected_features = ['Hirshfeld Heavy Atom Charge', 'Hirshfeld Carbon Charge', 'Hirshfeld Hydrogen Charge', 'ESP Heavy Atom Charge', 
                     'ESP Hydrogen Charge', 'NPA Hydrogen Charge', 'Mulliken Heavy Charge', 'Mulliken Hydrogen Charge', 'ATS0p', 'AATS2p', 'AATS4i', 'AATSC2d', 
                     'MATS3s', 'MATS5p', 'GATS2v', 'GATS4se', 'BCUTdv-1l', 'BCUTZ-1l', 'RPCG', 'AETA_beta']

#Loads validation dataset for borlation using the final reduced features 
unknownSubstrates=pd.read_csv('validation8-26-24.csv')

# Convert validation substrates Inchi's to Mordred and combine into Dataframe with atomic charges and JChem paramters
New_Substrate = unknownSubstrates['Substrate']
New_Substrate_mol_list = []
for inChi_New_Substrate in New_Substrate:
  New_Substrate_mol = Chem.MolFromInchi(inChi_New_Substrate)
  New_Substrate_mol_list.append(New_Substrate_mol)

New_Substrate_data = []
New_Substrate_data = calc.pandas(New_Substrate_mol_list)
New_Substrate_data = New_Substrate_data.apply(pd.to_numeric, errors='coerce')
New_Substrate_data.fillna(0, inplace=True)                                                                  
XnewSec = pd.concat((unknownSubstrates, New_Substrate_data), axis=1)
Xnew = XnewSec[selected_features]

val_tot = pd.DataFrame()

for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 200,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(data, "grouped", set_numbers)
    test_data = filter_rows_by_values2(data, "grouped", set_numbers)
   
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio' , axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio' , axis = 1)
    test_y = test_data['Product_Ratio']
   
    #Apply over-sampling to training set
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(training_X, training_y)    
    X_train = X_resampled[selected_features]

    #Random Forest Classifier
    rfc = RandomForestClassifier(n_estimators=800,max_depth=9)
    rfc.fit(X_train, y_resampled)
    
    #Evaluate the model on validation set
    ynew = rfc.predict(Xnew)
    validation_prediction_df = pd.DataFrame(ynew, columns = [(x)])
    validation_prediction_df.merge(validation_prediction_df, on=x)
    val_pred_T = validation_prediction_df.T
    val_tot = val_tot.append(val_pred_T)        

#Print the validation evaluations for model
unknownSubstrates_prod = unknownSubstrates['Product_Ratio']
total_val_results_transposed = val_tot.T
Val_results = pd.concat((unknownSubstrates_prod, total_val_results_transposed), axis=1)

#Write the results onto a CSV file 

Val_results.to_csv("chargeMordredEXP.csv", mode="a")

Val_results

100%|██████████| 81/81 [00:12<00:00,  6.34it/s]


,Product_Ratio,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
5,1,1,1,1,1,1,1,1,1,1,1
6,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0
8,1,1,1,1,1,1,1,1,1,1,1
9,0,0,0,0,0,0,0,0,0,0,0
